# VD-VAE

# Section 0 - Set-up 

In [ ]:
pip install pyts

     |████████████████████████████████| 2.5MB 7.9MB/s 


In [ ]:
! pip install kornia
!git clone https://www.github.com/EugenHotaj/pytorch-generative pytorch_generative_all 
!mv pytorch_generative_all/pytorch_generative pytorch_generative

     |████████████████████████████████| 286kB 7.8MB/s 
Cloning into 'pytorch_generative_all'...
remote: Enumerating objects: 1183, done.
remote: Counting objects: 100% (428/428), done.
remote: Compressing objects: 100% (330/330), done.
remote: Total 1183 (delta 303), reused 193 (delta 98), pack-reused 755
Receiving objects: 100% (1183/1183), 37.07 MiB | 30.03 MiB/s, done.
Resolving deltas: 100% (806/806), done.


In [ ]:
import torch
import torchvision
import torch.optim as optim
import argparse
import matplotlib
import torch.nn as nn
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from torch.nn import functional as F
import tensorflow as tf
from tqdm import tqdm
matplotlib.style.use('ggplot')

# Get things ready
import pytorch_generative as pg
#import pytorch_generative.models.vd_vae as vd_vae

# import function 
import numpy as np 
from os.path import join
from os import getcwd
import pandas as pd

from google.colab import drive
import sys

#import functions
import pyts
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
from pyts.image import GramianAngularField
from pyts.image import RecurrencePlot
from pyts.image import MarkovTransitionField

import numpy as np
from sklearn.model_selection import train_test_split

import gc

from sklearn.metrics import f1_score



In [ ]:
## set-up path
drive.mount('/content/drive')
sys.path.append('/content/drive/My Drive/Master in Data Science and Society/Master Thesis/Code & Data Set')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Section 1 - Pre-processing


## Functions

In [ ]:
# functions 
def preprocess_images(images):
  images = images.reshape((images.shape[0], 1, window_size,window_size))
  return images

def rolling_window(ts, window):
  """  
  Segreates times series to windows of size S 
  in rolling fashion 

  For example: 
  window0 = [x0, x1, x2]
  window1 = [x1, x2, x3]
  ...
  
  Arguments: 
  > ts = univariate time series 
  > window_size = int length of window
  """
  shape = ts.shape[:-1] + (ts.shape[-1] - window + 1, window)
  strides = ts.strides + (ts.strides[-1],)
  return np.lib.stride_tricks.as_strided(ts, shape=shape, strides=strides)
  
      # """References"""
      # [1]https://rigtorp.se/2011/01/01/rolling-statistics-numpy.html
      # [2]https://stackoverflow.com/questions/27852343/split-python-sequence-time-series-array-into-subsequences-with-overlap


In [ ]:
import pytorch_generative.models.vae.vd_vae as vd_vae
# Hyperparameter - unchanged for now

## Loss, Optimizer, sample 

def loss_fn(x, _, preds):
        preds, kl_div = preds
        recon_loss = F.mse_loss(preds, x, reduction="none")
        recon_loss = recon_loss.sum(dim=(1, 2, 3))
        elbo = recon_loss + kl_div
        log.append((recon_loss.mean(),kl_div.mean(),elbo.mean()))

        return {
            "recon_loss": recon_loss.mean(),
            "kl_div": kl_div.mean(),
            "loss": elbo.mean(),
        }

def sample_fn(model):
        sample = torch.sigmoid(model.sample(n_samples=16))
        return torch.where(
            sample < 0.5, torch.zeros_like(sample), torch.ones_like(sample)
        )


# Train VAE 

In [ ]:
#List of paths 
path_names_AdEx = ['twitterADVec_exchange-2_cpc_results',
                   'twitterADVec_exchange-2_cpm_results',
                   'twitterADVec_exchange-3_cpc_results',
                   'twitterADVec_exchange-3_cpm_results',
                   'twitterADVec_exchange-4_cpc_results',
                   'twitterADVec_exchange-4_cpm_results']

path_names_RealTraffic = ['twitterADVec_Twitter_volume_AAPL', 
                          'twitterADVec_Twitter_volume_AMZN',
                          'twitterADVec_Twitter_volume_CRM', 
                          'twitterADVec_Twitter_volume_CVS', 
                          'twitterADVec_Twitter_volume_FB', 
                          'twitterADVec_Twitter_volume_GOOG',
                          'twitterADVec_Twitter_volume_IBM', 
                          'twitterADVec_Twitter_volume_KO', 
                          'twitterADVec_Twitter_volume_PFE', 
                          'twitterADVec_Twitter_volume_UPS']

In [ ]:
eval_score = {}
n_epochs=10 #seems to overfit after 5 epochs
log_dir="/tmp/run"
n_gpus=0
device_id=0
debug_loader=None
log=[]
batch_size=32 
window_size=64

#set up data preprocessing
gasf = GramianAngularField(image_size=window_size, method='summation')# , sample_range=(0,1))


# stack config of model 
Child_IMGNET32_configs = [
      vd_vae.StackConfig(n_encoder_blocks=7, n_decoder_blocks=7),
      vd_vae.StackConfig(n_encoder_blocks=7, n_decoder_blocks=7),
      vd_vae.StackConfig(n_encoder_blocks=7, n_decoder_blocks=7),
      vd_vae.StackConfig(n_encoder_blocks=7, n_decoder_blocks=7),
      vd_vae.StackConfig(n_encoder_blocks=7, n_decoder_blocks=7),
      vd_vae.StackConfig(n_encoder_blocks=4, n_decoder_blocks=4),
    ]

DEFAULT_MODEL = [
      vd_vae.StackConfig(n_encoder_blocks=1, n_decoder_blocks=1),
      vd_vae.StackConfig(n_encoder_blocks=1, n_decoder_blocks=1),
      vd_vae.StackConfig(n_encoder_blocks=1, n_decoder_blocks=1),
      vd_vae.StackConfig(n_encoder_blocks=1, n_decoder_blocks=1),
      vd_vae.StackConfig(n_encoder_blocks=1, n_decoder_blocks=1),
      vd_vae.StackConfig(n_encoder_blocks=1, n_decoder_blocks=1),
    ]

for name in path_names_AdEx: 

  #print path
  print(f'Dataset: {name}')

  #build path realAdExchange
  if name in path_names_AdEx: 
    path = f'/content/drive/My Drive/Master in Data Science and Society/Master Thesis/Code & Data Set/labels/realAdExchange/{name}.csv'
  else:
    path = f'/content/drive/My Drive/Master in Data Science and Society/Master Thesis/Code & Data Set/labels/realTweets/{name}.csv'

  print(path)

  #load data set
  df = pd.read_csv(path)
  ts = np.array(df['value'])

  #GAF
  ts_windows = rolling_window(ts, window_size)
  
  ## set up list storing img of ts window
  ts_windows_img = []

  #convert windows to img
  for ts_window in ts_windows:
    X_gasf = gasf.fit_transform(ts_window.reshape(1,-1))
    ts_windows_img.append(X_gasf[0])

  # convert to img list to tensor
  tensor_img = torch.Tensor(ts_windows_img)

  # Data loader set up 
  X_train, X_val, _, _ = train_test_split(tensor_img, 
                                          tensor_img,
                                          test_size=0.1, 
                                          random_state=42)

  print('Pre-process complete')

  ## Dataset GAF
  train = torch.utils.data.DataLoader(preprocess_images(X_train),
                                            batch_size=batch_size,
                                            shuffle=False)

  val = torch.utils.data.DataLoader(preprocess_images(X_val),
                                            batch_size=batch_size ,
                                            shuffle=False)

  test = torch.utils.data.DataLoader(preprocess_images(tensor_img),
                                            batch_size=batch_size ,
                                            shuffle=False)

  model = vd_vae.VeryDeepVAE(
            in_channels=1,
            out_channels=1,
            input_resolution=64,
            stack_configs=Child_IMGNET32_configs,
            latent_channels=4, #4  #set low since time series data less informative
            hidden_channels=128, #16
            bottleneck_channels=32, #8
    )

  # loss 
  optimizer = optim.Adam(model.parameters(), lr=0.000015)

  #model trainer
  model_trainer = pg.trainer.Trainer(
            model=model,
            loss_fn=loss_fn,
            optimizer=optimizer,
            train_loader=train,
            eval_loader=val,
            sample_epochs=1,
            sample_fn=sample_fn,
            log_dir=log_dir,
            n_gpus=n_gpus,
            device_id=device_id,
          )
  model_trainer.interleaved_train_and_eval(n_epochs, restore=False)

  print('Model training complete')

  # save/ load model 
  torch.save(model, f'/content/drive/My Drive/Master in Data Science and Society/Master Thesis/Code & Data Set/test_results/models/test{name}.h5')

  # evaluate 
  model_eval = model.train(False)

  #reconstruction
  MSEs_of_inputs = []

  for b in test: 
                  gc.collect()
                  x = model_eval(b)
                  x = x[0].detach()

                  #MSE of inputs of batch
                  sq_diff = torch.square(x - b)
                  Batch_MSE = 0.5*torch.sum(sq_diff, [1,2,3])

                  MSEs_of_inputs.append(Batch_MSE)
                  gc.collect()

      # convert to tensor 
  MSEs_of_inputs = torch.cat(MSEs_of_inputs)
  print('Reconstruction complete')

        # Threshold 
  std, mean = torch.std_mean(MSEs_of_inputs) # returns standard-deviation and mean

      
  y_pred = []
  df['pred'] = 0
  for ind, r_error in enumerate(MSEs_of_inputs): 
          
          # Anomaly detection
          if r_error > (mean + 2*std) or r_error < (mean - 2*std):
                
              #print('Anomaly')
              df['pred'][ind+(64-1)] = 1 #w is width set initially -1 due to Xue et al implementation
              y_pred.append(1)

          else: 
              #print('Normal')
              df['pred'][ind+(64-1)] = 0
              y_pred.append(0)

  print('Anomalies specified')
      #save df as csv
  df.to_csv(f'/content/drive/My Drive/Master in Data Science and Society/Master Thesis/Code & Data Set/test_results/datasets/test{name}.csv')
      
      #true labels
  y_true = df['label'][63:-64].tolist()

  eval_score[f'Test{name}'] = f1_score(y_true, y_pred) #assoiate with settings 
        
  print(f1_score(y_true, y_pred))
  tn, fp, fn, tp = confusion_matrix(df['label'][(w-1):-w], df['pred'][(w-1):-w]).ravel()
  print(f'Confusion matrix: TN: {tn}, FP{fp}, FN{fn}, TP{tp}')
  print(f'====')

In [ ]:
y_true = df['label'][63:-64].tolist()
f1_score(y_true, y_pred)

0.0